### Tool calling

This is just a prototyping notebook for tool calling with langchain and ollama

In [23]:
#from langchain_experimental.llms.ollama_functions import OllamaFunctions
from langchain_ollama import ChatOllama, OllamaLLM
from langchain.prompts import ChatPromptTemplate
from langchain.schema import SystemMessage, HumanMessage, AIMessage
import logging
from langchain.chat_models import init_chat_model

from tool_functions import *

In [2]:
model = init_chat_model(
    "llama3.2",
    model_provider="ollama",
    temperature=0.5,
    max_tokens=100,
)

In [3]:
#model = ChatOllama(model="llama3.2", format="json", temperature=0)
model_with_tools = model.bind_tools(tools=get_tools())

In [4]:
messages = [HumanMessage(content="What is the current electricity price in Finland?")]
response = model_with_tools.invoke(messages)
print(response.tool_calls)

[{'name': 'fetch_current_electricity_price', 'args': {}, 'id': 'c2241cf3-a86a-4a5c-9935-d0d811b3f1ea', 'type': 'tool_call'}]


In [5]:
response.tool_calls

[{'name': 'fetch_current_electricity_price',
  'args': {},
  'id': 'c2241cf3-a86a-4a5c-9935-d0d811b3f1ea',
  'type': 'tool_call'}]

In [6]:
messages = [
    SystemMessage(content="""You are a helpful assistant and you can call functions to get information.
    You can call multiple functions if necessary.
                  """
                  ),
    HumanMessage(content="What is the current date? Also what is current time?"),
]

In [7]:
ai_message = model_with_tools.invoke(messages)
messages.append(ai_message)
for tool_call in ai_message.tool_calls:
    selected_tool = tool_map[tool_call["name"].lower()]
    tool_msg = selected_tool.invoke(tool_call)
    messages.append(tool_msg)

messages

[SystemMessage(content='You are a helpful assistant and you can call functions to get information.\n    You can call multiple functions if necessary.\n                  ', additional_kwargs={}, response_metadata={}),
 HumanMessage(content='What is the current date? Also what is current time?', additional_kwargs={}, response_metadata={}),
 AIMessage(content='', additional_kwargs={}, response_metadata={'model': 'llama3.2', 'created_at': '2025-04-27T19:42:01.314577523Z', 'done': True, 'done_reason': 'stop', 'total_duration': 909250109, 'load_duration': 17007963, 'prompt_eval_count': 895, 'prompt_eval_duration': 140000000, 'eval_count': 28, 'eval_duration': 316000000, 'model_name': 'llama3.2'}, id='run-3020a388-52c5-4362-be8f-3be88dce5598-0', tool_calls=[{'name': 'get_current_date', 'args': {}, 'id': '000ad38d-4401-4950-ad42-1cb0989113b8', 'type': 'tool_call'}, {'name': 'get_current_time', 'args': {}, 'id': '63a4fc11-78e5-4c55-9ba6-1b907c3d4d90', 'type': 'tool_call'}], usage_metadata={'inp

In [8]:
response = model_with_tools.invoke(messages)
print(response)

content='The current date is April 27, 2025. The current time is 10:42 PM.' additional_kwargs={} response_metadata={'model': 'llama3.2', 'created_at': '2025-04-27T19:42:05.661919914Z', 'done': True, 'done_reason': 'stop', 'total_duration': 212615509, 'load_duration': 15976174, 'prompt_eval_count': 140, 'prompt_eval_duration': 4000000, 'eval_count': 23, 'eval_duration': 191000000, 'model_name': 'llama3.2'} id='run-6810b470-cf5c-4aab-985c-92cc05e4c1f7-0' usage_metadata={'input_tokens': 140, 'output_tokens': 23, 'total_tokens': 163}


### Using prompt templates

In [107]:
system_message = """
You are a helpful assistant and you can call functions to get information.
If you need to use the outputs of functions as inputs to other functions, you can do that
also. You can call multiple functions if necessary. Explain what you are doing in the process.
You can also ask clarifying questions if you need more information.
"""

prompt = ChatPromptTemplate(
    [
        ("system", system_message),
        ("human", "{user_input}"),        
    ]
)


In [55]:
type(prompt)

langchain_core.prompts.chat.ChatPromptTemplate

In [ ]:
type(res)

list

In [51]:
model_with_tools.invoke(res)

INFO:httpx:HTTP Request: POST http://127.0.0.1:11434/api/chat "HTTP/1.1 200 OK"


AIMessage(content='', additional_kwargs={}, response_metadata={'model': 'llama3.2', 'created_at': '2025-04-28T09:00:57.376071878Z', 'done': True, 'done_reason': 'stop', 'total_duration': 1970160616, 'load_duration': 16041664, 'prompt_eval_count': 934, 'prompt_eval_duration': 74000000, 'eval_count': 28, 'eval_duration': 1877000000, 'model_name': 'llama3.2'}, id='run-89a48ac3-3f1b-446c-a273-ff15f4bbc1d2-0', tool_calls=[{'name': 'get_current_date', 'args': {}, 'id': '7d08fb8e-a693-42c8-a9a9-27cfe9f496cb', 'type': 'tool_call'}, {'name': 'get_current_time', 'args': {}, 'id': 'f4312f22-828e-4838-92c5-37ebc839b33f', 'type': 'tool_call'}], usage_metadata={'input_tokens': 934, 'output_tokens': 28, 'total_tokens': 962})

In [108]:
res=prompt.invoke({"user_input": "What is the current date? Also what is current time?"})
res.messages

[SystemMessage(content='\nYou are a helpful assistant and you can call functions to get information.\nIf you need to use the outputs of functions as inputs to other functions, you can do that\nalso. You can call multiple functions if necessary. Explain what you are doing in the process.\nYou can also ask clarifying questions if you need more information.\n', additional_kwargs={}, response_metadata={}),
 HumanMessage(content='What is the current date? Also what is current time?', additional_kwargs={}, response_metadata={})]

In [109]:
res=(prompt | model_with_tools).invoke({"user_input": "What is the current date? Also what is current time?"})
res

INFO:httpx:HTTP Request: POST http://127.0.0.1:11434/api/chat "HTTP/1.1 200 OK"


AIMessage(content='', additional_kwargs={}, response_metadata={'model': 'llama3.2', 'created_at': '2025-04-28T09:31:55.740898279Z', 'done': True, 'done_reason': 'stop', 'total_duration': 20397608069, 'load_duration': 15226177, 'prompt_eval_count': 937, 'prompt_eval_duration': 18497000000, 'eval_count': 28, 'eval_duration': 1883000000, 'model_name': 'llama3.2'}, id='run-4d96106f-2720-4087-a375-9950556b8e13-0', tool_calls=[{'name': 'get_current_date', 'args': {}, 'id': 'f836c174-4c8c-41fa-9c92-5537dd58b0e8', 'type': 'tool_call'}, {'name': 'get_current_time', 'args': {}, 'id': 'e8cec759-d24c-4e1b-ba06-0136dee6a56b', 'type': 'tool_call'}], usage_metadata={'input_tokens': 937, 'output_tokens': 28, 'total_tokens': 965})

In [110]:
res.tool_calls

[{'name': 'get_current_date',
  'args': {},
  'id': 'f836c174-4c8c-41fa-9c92-5537dd58b0e8',
  'type': 'tool_call'},
 {'name': 'get_current_time',
  'args': {},
  'id': 'e8cec759-d24c-4e1b-ba06-0136dee6a56b',
  'type': 'tool_call'}]

In [99]:

# Set up logging
logging.basicConfig(level=logging.INFO)
logger = logging.getLogger(__name__)
# Set up the function to call the model
def call_model(input_text, prompt, verbose=False):
    '''
    Calls the AI model with the provided input text and returns the response.

    Args:
        prompt (ChatPromptTemplate): The prompt template to use for the model.
        input_text (str): The input text to send to the AI model.

    Returns:
        str: The response from the AI model.
        prompt (ChatPromptTemplate): The updated prompt with the model response.
    '''

    chain = prompt | model_with_tools
    try:
        # Call the model with the input text
        response = chain.invoke({"user_input": input_text})
        #prompt.messages.append(response)
        if verbose: logger.info(f"Model response: {response}")
        
        # Check if the response contains tool calls
        if response.tool_calls:
            for tool_call in response.tool_calls:
                selected_tool = tool_map[tool_call["name"].lower()]
                tool_msg = selected_tool.invoke(tool_call)
                if verbose: logger.info(f"Tool response: {tool_msg}")
                # Append the tool response to the messages
                prompt.messages.append(tool_msg)
            # call the model again with the updated messages
            return call_model(input_text, prompt)
        
        if verbose: logger.info("No tool calls in the response.")
        prompt.messages.append(response)
        return response, prompt
    
    except Exception as e:
        logger.error(f"Error calling model: {e}")
        return None, prompt

In [100]:
input_text = "if x = 24-18, what is x divided by 3?"
response, prompt = call_model(input_text, prompt=prompt)
if response:
    print(f"Response: {response}")
else:
    print("Failed to get a response from the model.")

INFO:httpx:HTTP Request: POST http://127.0.0.1:11434/api/chat "HTTP/1.1 200 OK"
INFO:httpx:HTTP Request: POST http://127.0.0.1:11434/api/chat "HTTP/1.1 200 OK"


Response: content='To find the value of x and then divide it by 3, we can follow these steps:\n\n1. Calculate the value of x: x = 24 - 18\nx = 6\n\n2. Divide x by 3: 6 / 3\n6 / 3 = 2.0\n\nTherefore, x divided by 3 is 2.0.' additional_kwargs={} response_metadata={'model': 'llama3.2', 'created_at': '2025-04-28T09:30:16.820614534Z', 'done': True, 'done_reason': 'stop', 'total_duration': 5022938177, 'load_duration': 15412378, 'prompt_eval_count': 148, 'prompt_eval_duration': 470000000, 'eval_count': 79, 'eval_duration': 4535000000, 'model_name': 'llama3.2'} id='run-e5c5b85e-6519-4335-8f01-e023a40f021b-0' usage_metadata={'input_tokens': 148, 'output_tokens': 79, 'total_tokens': 227}


In [103]:
response.content.splitlines()

['To find the value of x and then divide it by 3, we can follow these steps:',
 '',
 '1. Calculate the value of x: x = 24 - 18',
 'x = 6',
 '',
 '2. Divide x by 3: 6 / 3',
 '6 / 3 = 2.0',
 '',
 'Therefore, x divided by 3 is 2.0.']